In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess

family_group = "3.Satyrine"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            scaffold = coordinate_line_split[1]
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "16":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Bicyclus_anynana
GCF_947172395.1_ilBicAnyn1.1_genomic.fna
8759179 8759216
0 2
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MEDDGDDKDDFK
>Bicyclus_anynana_Exon_1_NC_069110.1_8759179_8759216_left_0_right_2
ATGGAGGACGATGGGGATGACAAAGATGATTTCAAAAG
8758140 8758288
1 1
Getting Gene
>Bicyclus_anynana_Exon_2_NC_069110.1_8758140_8758288_left_1_right_1
gAGGACACGTAATCTTAGTGAAAAGAAAAGACGTGATCAATTTAATACGCTTGTTCATGAACTGGGAGCAATGATCTCATCAAATAACAGAAAAATGGATAAATCAACAATTCTCCGAACAACCATATACTTCTTAAAGAATCATAATG
8757713 8757894
2 0
Getting Gene
>Bicyclus_anynana_Exon_3_NC_069110.1_8757713_8757894_left_2_right_0
AAATAACAGCACGAGCAAGAGCCCACGATGTGCAAGACGATTGGAAACCTGCTTTCTTGACTAACGAAGAGTTCACATATTTACTACTTGAGGCTTTGGAGGGTTTTGTCATAGTATTTTCAGCAAATGGACGCATCATTTATGTCTCTGAAAGTGTTGCATCTCTATTAGGCCATAACCcg
8757213 8757330
0 1
Getting Gene
>Bicyclus_anynana_Exon_4_NC_069110.1_8757213_8757330_left_0_right_1
aatgatattattaataaaaatatatatgatttaGTATATAAAGATGATCGCTGTAATCTTTATGGATTACTACAAAATCCTGATACATCAGTGCATGCAGCGAAAAACGACAATGCAG
8753106 8753201
2 1
Getting Gene
>Bicyclus_anynana_Exon_5_NC_069110.1_8753106_8753201_left_2_right_1
acAATGAGATAAATTTCCAATGTC

Elymnias_hypermnestra
GCA_015832385.1_UofC_Ehyp_1.0_genomic.fna
960613 960650
0 2
Getting Gene
MEDDGDDKDDSK
>Elymnias_hypermnestra_Exon_1_JACTOM010000009.1_960613_960650_left_0_right_2
ATGGAGGACGATGGAGACGACAAAGATGACTCTAAAAG
959018 959166
1 1
Getting Gene
>Elymnias_hypermnestra_Exon_2_JACTOM010000009.1_959018_959166_left_1_right_1
GAGGACTCGTAACCTAAGTGAGAAAAGGCGAAGGGATCAATTTAATATGCTTGTCAATGAGCTTGGTACAATGGTTACAGTAGGTAACAGAAAAATGGACAAATCTACAGTCCTCAAATCTACCATAGCATTTCTAAGGCATCACAATG
958752 958933
2 0
Getting Gene
>Elymnias_hypermnestra_Exon_3_JACTOM010000009.1_958752_958933_left_2_right_0
aAATAACTATGCGATCAAGAACATATGATGTACAAGAAAACTGGAAACCAGCTTTCCTTTCAAATGAGGAGTTTACATATTTAGTACTTGAAGCACTTGAAGGATTTGTAATGGTGTTTTCTGTAACTGGTAGCATATATTATGTATCTGAAAGTATTACATCACTATTAGGCCACAATTCG
958181 958298
0 1
Getting Gene
>Elymnias_hypermnestra_Exon_4_JACTOM010000009.1_958181_958298_left_0_right_1
AATGATATAGTAAACAAAAGTATATTTGACTTGATATTTGAAGAGGAGAGGCCTAGTTTGTTACAACTCCTCCAAAATTCTAATTCTTTCATTAATTCCTCAGAATGGTTTCTTGCag
